# What is this?
A notebook to experiment with the BVG (Berlin public transport) API.

https://v6.bvg.transport.rest/
rate limit: 100 req/min

In [28]:
# Find Kuno Fischer and Amtsgericht station ID (the latter only for direction)
#!pip install requests
import requests

# API URL
url = "https://v6.bvg.transport.rest/stops"

# Make the GET request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()  # Parse JSON response
else:
    print(f"Error: {response.status_code}")

for d in data:
    if "Kuno-Fischer" in d["name"]:
        print(f"ID for '{d['name']}': '{d['id']}'")
    if "Amtsgerichtsplatz" in d["name"]:
        print(f"ID for '{d['name']}': '{d['id']}'")
    if "ICC" in d["name"]:
        print(f"ID for '{d['name']}': '{d['id']}'")

ID for 'S Messe Nord/ICC': 'de:11000:900024106'
ID for 'S Messe Nord/ICC': 'de:11000:900024106::4'
ID for 'Messegelände/ICC': 'de:11000:900025552::1'
ID for 'S Messe Nord/ICC': 'de:11000:900024106::1'
ID for 'Kuno-Fischer-Str.': 'de:11000:900024151::1'
ID for 'S Messe Nord/ICC': 'de:11000:900024106::3'
ID for 'Amtsgerichtsplatz': 'de:11000:900024104::1'
ID for 'Kuno-Fischer-Str.': 'de:11000:900024151::2'
ID for 'S Messe Nord/ICC': 'de:11000:900024106::2'
ID for 'Amtsgerichtsplatz': 'de:11000:900024104::2'
ID for 'Messegelände/ICC': 'de:11000:900025552::2'
ID for 'Amtsgerichtsplatz': 'de:11000:900024104::3'
ID for 'S Messe Nord/ICC': 'de:11000:900024106::5'
ID for 'S Messe Nord/ICC': 'de:11000:900024106::6'
ID for 'S Messe Nord/ICC': 'de:11000:900024106::7'


In [ ]:
# API endpoint
url = "https://v6.bvg.transport.rest/stops/900024151/departures"

# Query parameters (customize as needed)
params = {
    # "direction": "900024104",  # Optional: Filter departures by a specific direction
    "direction": "900024106",
    "duration": 30,  # Show departures for the next 10 minutes
    "remarks": True,  # Include warnings and hints
    "language": "en",  # Language of the results
    "pretty": True,  # Pretty-print JSON responses
}

# Send GET request
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()  # Parse JSON response
    print(data)  # Print or process the data as needed
else:
    print(f"Error: {response.status_code} - {response.text}")

{'departures': [{'tripId': '1|53000|0|86|30062025', 'stop': {'type': 'stop', 'id': '900024151', 'name': 'Kuno-Fischer-Str. (Berlin)', 'location': {'type': 'location', 'id': '900024151', 'latitude': 52.506613, 'longitude': 13.289778}, 'products': {'suburban': False, 'subway': False, 'tram': False, 'bus': True, 'ferry': False, 'express': False, 'regional': False}}, 'when': '2025-06-30T11:11:00+02:00', 'plannedWhen': '2025-06-30T11:11:00+02:00', 'delay': None, 'platform': None, 'plannedPlatform': None, 'prognosisType': None, 'direction': 'Heerstr./Nennhauser Damm', 'provenance': None, 'line': {'type': 'line', 'id': 'de-vbb-11000000-bus-m49', 'fahrtNr': '135395', 'name': 'M49', 'public': True, 'adminCode': 'BVB---', 'productName': 'Bus', 'mode': 'bus', 'product': 'bus', 'operator': {'type': 'operator', 'id': 'berliner-verkehrsbetriebe', 'name': 'Berliner Verkehrsbetriebe'}}, 'remarks': [], 'origin': None, 'destination': {'type': 'stop', 'id': '900037211', 'name': 'Heerstr./Nennhauser Damm 

In [30]:
data["departures"][0]

{'tripId': '1|53000|0|86|30062025',
 'stop': {'type': 'stop',
  'id': '900024151',
  'name': 'Kuno-Fischer-Str. (Berlin)',
  'location': {'type': 'location',
   'id': '900024151',
   'latitude': 52.506613,
   'longitude': 13.289778},
  'products': {'suburban': False,
   'subway': False,
   'tram': False,
   'bus': True,
   'ferry': False,
   'express': False,
   'regional': False}},
 'when': '2025-06-30T11:11:00+02:00',
 'plannedWhen': '2025-06-30T11:11:00+02:00',
 'delay': None,
 'platform': None,
 'plannedPlatform': None,
 'prognosisType': None,
 'direction': 'Heerstr./Nennhauser Damm',
 'provenance': None,
 'line': {'type': 'line',
  'id': 'de-vbb-11000000-bus-m49',
  'fahrtNr': '135395',
  'name': 'M49',
  'public': True,
  'adminCode': 'BVB---',
  'productName': 'Bus',
  'mode': 'bus',
  'product': 'bus',
  'operator': {'type': 'operator',
   'id': 'berliner-verkehrsbetriebe',
   'name': 'Berliner Verkehrsbetriebe'}},
 'remarks': [],
 'origin': None,
 'destination': {'type': 'stop

In [48]:
import pandas as pd
import datetime
from zoneinfo import ZoneInfo

berlin = ZoneInfo("Europe/Berlin")


def get_stop_data():
    # API endpoint
    url = "https://v6.bvg.transport.rest/stops/900024151/departures"

    # Query parameters (customize as needed)
    params = {
        "when": (datetime.datetime.now(berlin) + datetime.timedelta(minutes=-5)).isoformat(),
        # "direction": "900024104",  # Optional: Filter departures by a specific direction
        "direction": "900024106",
        "duration": 25,  # Show departures for the next 25 minutes
        "remarks": True,  # Include warnings and hints
        "language": "en",  # Language of the results
        "pretty": True,  # Pretty-print JSON responses
    }

    # Send GET request
    response = requests.get(url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()  # Parse JSON response
    else:
        print(f"Error: {response.status_code} - {response.text}")

    result = {
        "type": [],
        "line": [],
        "departure": [],
        "delay": [],
        "direction": [],
        "cancelled": [],
    }

    for bus in data["departures"]:
        assert bus["stop"]["products"]["bus"], "The heck? There can only be busses."

        result["type"].append("Bus")
        result["line"].append(bus["line"]["name"])
        result["departure"].append(
            datetime.datetime.fromisoformat(bus["plannedWhen"]).strftime("%H:%M:%S")
        )
        delay = 0 if bus["delay"] is None else bus["delay"]
        delay /= 60
        result["delay"].append(delay)
        result["direction"].append(bus["direction"])
        if "cancelled" in bus:
            result["cancelled"].append(bus["cancelled"])
        else:
            result["cancelled"].append(False)

    updated_at = datetime.datetime.fromtimestamp(data["realtimeDataUpdatedAt"]).strftime("%H:%M:%S")

    return updated_at, pd.DataFrame(result)

In [49]:
t, r = get_stop_data()
print(t)
print(r)

11:06:32
  type line departure  delay                 direction  cancelled
0  Bus  M49  11:11:00    0.0  Heerstr./Nennhauser Damm      False
1  Bus  M49  11:21:00    0.0        Staaken, Reimerweg      False
2  Bus  M49  11:31:00    0.0  Heerstr./Nennhauser Damm      False
